##### 问题:
我们已经把装饰器添加到一个函数上了，但是想“撤销”它，访问未经过包装的那个
原始函数。

##### 解决方案:
假设装饰器的实现中已经使用了@wraps（参见 9.2 节），一般来说我们可以通过访问
__wrapped__属性来获取对原始函数的访问。示例如下：

In [9]:
import time
from functools import wraps
def somedecorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper


@somedecorator
def add(x, y):
    return x + y
orig_add = add.__wrapped__
orig_add(3, 4)

7

直接访问装饰器背后的那个未包装过的函数对于调试、反射（introspection，也有译为
'自省'）以及其他一些涉及函数的操作是很有帮助的。但是，本节讨论的技术只有在
实现装饰器时利用 functools 模块中的@wraps 对元数据进行了适当的拷贝，或者直接设
定了__wrapped__属性时才有用。

如果有多个装饰器已经作用于某个函数上了，那么访问__wrapped__属性的行为目前是
未定义的，应该避免这种情况。在 Python 3.3 中，这么做会绕过所有的包装层。例如，
假设有如下的代码：(3.9版本中只绕过了一个包装层，可能是bug修复了)

In [10]:
from functools import wraps
def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper
def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper
    
@decorator1
@decorator2
def add(x, y):
    return x + y

当调用装饰过的函数以及通过__wrapped__属性调用原始函数时就会出现这样的
情况：

In [11]:
add(2, 3)

Decorator 1
Decorator 2


5

In [12]:
add.__wrapped__(2, 3)   #可能是bug修复了,只绕过了一个包装层

Decorator 2


5

最后但同样重要的是，请注意并不是所有的装饰器都使用了@wraps，因此有些装饰器的行
为可能与我们期望的有所区别。特别是，由内建的装饰器@staticmethod 和@classmethod创建的描述符（descriptor）对象并不遵循这个约定（相反，它们会把原始函数保存在__func__属性中）。所以，具体问题需要具体分析，每个人遇到的情况可能不同。